# Facial Recognition Deep Learning

### Installing Dependencies

In [ ]:
# %pip install opencv-python matplotlib tensorflow-macos

# tensorflow~=2.6.0 and tensorflow-gpu==2.6.0 for non mac users


## Importing Libraries

In [ ]:
%%time
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
import uuid
# tensorflow functional apis
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten

In [ ]:
# Input & Output Model Look Like
#Model(input=[inputimgage, verificationImage], output=[1,0])


### GPU GROWTH (LIMIT GPU)
##### DON'T USE THIS COMMAND IF YOU DON'T HAVE GPU ON YOUR DEVICE

In [ ]:
# Avoid OOM errors by setting GPU memory consumption growth
gpus = tf.config.experimental.list_physical_devices('GPU')

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


#### Folder Structure 

In [ ]:
# specifying folder path
POSITIVE_PATH = os.path.join('data', 'positive')
NEGATIVE_PATH = os.path.join('data', 'negative')
ANCHOR_PATH = os.path.join('data', 'anchor')
# creating the folders
# os.makedirs(POSITIVE_PATH)
# os.makedirs(NEGATIVE_PATH)
# os.makedirs(ANCHOR_PATH)


## Collect Dataset (Positive and Anchor)

In [ ]:
# decompress Tar GZ file
# !tar -xf lfw.tgz


In [ ]:
# Move Images to Data/Negative
# for dir in os.listdir('lfw'):
#     # to get the inside folder paths
#     for file in os.listdir(os.path.join('lfw',dir)):
#         exact_path = os.path.join('lfw',dir,file)
#         new_path = os.path.join(NEGATIVE_PATH,file)
#         os.replace(exact_path,new_path)


In [ ]:
# !rm -rf lfw


## Collect Positive and Anchor Class

### VideoCapture CV2 

In [ ]:
# Establish a connection to the webcam
cap = cv2.VideoCapture(0)

#resolution set to base level, 640x480, 1280x720
cap.set(3, 640)
cap.set(4, 480)

while cap.isOpened():
    ret, frame = cap.read()

    # Cutting down frame to 250x250 px (last we have used (:) for getting all the colors)
    frame = frame[120:120+250, 200:200+250, :]

    # Collect anchors
    if cv2.waitKey(1) & 0xFF == ord('a'):
        # Create a Unique file Path
        img_name = os.path.join(ANCHOR_PATH, f"{uuid.uuid1()}.jpg")
        # write image to anchor folder
        cv2.imwrite(img_name, frame)

    # Collect Positive
    if cv2.waitKey(1) & 0xFF == ord('p'):
        # Create a Unique file Path
        img_name = os.path.join(POSITIVE_PATH, f"{uuid.uuid1()}.jpg")
        # Write image to positive folder
        cv2.imwrite(img_name, frame)

    # Display the resulting frame
    cv2.imshow('Face Capture', frame)

    # Press Q on keyboard to stop recording
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
# When everything done, release the video capture and video write objects
cap.release()
cv2.waitKey(1)
# Closes all the frames
cv2.destroyAllWindows()
cv2.waitKey(1)

## Loading and Preprocessing Images

#### Get Image Directories

In [ ]:
%%time
# Here Collect image from a specific path contain .jpg extension files upto 300
anchor = tf.data.Dataset.list_files(ANCHOR_PATH+'/*.jpg').take(300)
positive = tf.data.Dataset.list_files(POSITIVE_PATH+'/*.jpg').take(300)
negative = tf.data.Dataset.list_files(NEGATIVE_PATH+'/*.jpg').take(300)

In [ ]:
dir_test = anchor.as_numpy_iterator()

In [ ]:
dir_test.next()